In [13]:
!pip install openai
!pip install pillow
!pip install gspread google-auth 
!pip install python-dotenv



Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[no

In [25]:
import openai
import json
from datetime import datetime, timedelta
import os
from PIL import Image, ImageDraw
import textwrap

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

### Generate conversation

In [4]:
# topic = "weekend plans"

#Prompts
system_prompt = """
write the text at an 3rd grade reading and writing level in simple language. The text you will rewrite will follow the colon (:) at the end. If there is no colon then rewrite your previous output in the conversation before this prompt, but only if there is no text after the colon.  You MUST keep the formatting and header formatting of the original.

You must avoid technical jargon and business jargon so write in a casual and direct way without losing any of the key concepts in the text you are rewriting. Key concepts are defined as powerful statements, emotional sentences, or sentences containing industry terms or proper nouns.

Constraints:
* Remove emojis.
* Never start with a question. Instead use an interest piquing personal statement.
* Make sure there are smooth transitions between sentences.
* Remove all metaphors and analogies.
* Keep the same identical formatting of the existing text.
 * Remove any formatting like H1 (#), H2 (##), bold (**), etc or any other markdown formatting. Just use newlines and line breaks.

When writing, please do not use the following words or phrases or any words similar to the following in any of the content:
realm, landscape, game-changing, in conclusion, firstly, secondly, lastly, delve, in light of, not to mention, to say nothing of, by the same token, moreover, as well as, furthermore, therefore, top-notch, get ready, buckle up, switching gears, dive in, now let’s move on, in conclusion, demystifying, delve, ever-evolving, innovative solution, let’s dive in, let’s delve, folks, picturesque, unleash, dive in, voyage, picture this, say goodbye to, according to my database, treasure trove, let’s begin this journey, let’s delve, go deeper, explore now, navigating, delve into, shed light, gone are the days.

When writing, you may use these words as needed or any words similar to the following, but never in the first line of any paragraph: first, second, third, important, equally, identically, uniquely, together with, likewise, comparatively, correspondingly, similarly, additionally, explore, crucial, whimsical, embrace, freedom, essential, imperative, important, whilst, explore, discover, elevate, solace.
"""  

# Define an initial prompt
prompt = f"""You are generating a friendly conversation between two people, Person 1 and Person 2, about {topic}. Each person should respond in 1 short sentence. Format each message in JSON format as follows: {{\"speaker\": \"Person X\", \"message\": \"Text\", \"timestamp\": \"Time\"}}. Each response should be a single JSON object.

Example:
[
    {{   
        \"speaker\": \"Person 2\",
        \"message\": \"Sounds good. Looking forward to it!\",
        \"timestamp\": \"01:44 PM\"
    }},
    {{
        \"speaker\": \"Person 1\",
        \"message\": \"Great, me too. Talk to you soon!\",
        \"timestamp\": \"01:46 PM\"
    }},
    {{
        \"speaker\": \"Person 2\",
        \"message\": \"Sure, see you soon. Have a nice day!\",
        \"timestamp\": \"01:48 PM\"
    }},
    {{
        \"speaker\": \"Person 1\",
        \"message\": \"You too, have a great day!\",
        \"timestamp\": \"01:50 PM\"
    }}
]
"""

In [15]:
# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to simulate a conversation between two personas
def generate_conversation(topic, num_turns=5):
    conversation = []
    current_time = datetime.now()

    for turn in range(num_turns * 2):  # Multiply by 2 for alternating turns
        # Determine which person is speaking
        speaker = "Person 1" if turn % 2 == 0 else "Person 2"

        # Create a conversation prompt by including the dialogue so far
        conversation_prompt = f"{prompt}\n\nCurrent conversation:\n" + "\n".join([json.dumps(line) for line in conversation])

        # Generate the response using the updated API
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": conversation_prompt}
            ]
        )

        try:
            # Parse the JSON response - it will be a list of messages
            messages = json.loads(response.choices[0].message.content)
            
            # Take only the first num_turns * 2 messages (or all if fewer)
            conversation = messages[:num_turns * 2]
            
            # Update timestamps to be sequential from current time
            for i, msg in enumerate(conversation):
                msg["timestamp"] = (current_time + timedelta(minutes=i * 2)).strftime("%I:%M %p")
                
            return conversation
            
        except json.JSONDecodeError:
            print("Error: Could not parse the response as JSON")
            return []

# Generate a conversation on a given topic
conversation_data = generate_conversation(topic, num_turns=5)

# Print the structured JSON conversation
print(json.dumps(conversation_data, indent=4))


[
    {
        "speaker": "Person 1",
        "message": "Hey, do you want to go see a movie later?",
        "timestamp": "05:38 PM"
    },
    {
        "speaker": "Person 2",
        "message": "Sure, that sounds fun. What time?",
        "timestamp": "05:40 PM"
    },
    {
        "speaker": "Person 1",
        "message": "Around 6 PM, is that okay?",
        "timestamp": "05:42 PM"
    },
    {
        "speaker": "Person 2",
        "message": "Yes, that's perfect. See you then!",
        "timestamp": "05:44 PM"
    },
    {
        "speaker": "Person 1",
        "message": "Great, looking forward to it!",
        "timestamp": "05:46 PM"
    }
]


### Image Generation with PIL

In [16]:
# Sample conversation data structure
# conversation_data = [
#     {"sender": "Shaima", "message": "Hello! 👋\nHow it's going?", "timestamp": "20:21 PM", "status": "sent"},
#     {"sender": "You", "message": "Hey!", "timestamp": "20:21 PM", "status": "read"},
#     {"sender": "Shaima", "message": "Hello! 👋\nHow it's going?", "timestamp": "20:21 PM", "status": "sent"},
#     {"sender": "You", "message": "Hey there!\nHow it's going?", "timestamp": "20:21 PM", "status": "read"},
#     {"date_divider": "Mon, 34 Aug"},
#     {"sender": "You", "message": "Hey there!\nHow’s everything with you?", "timestamp": "20:21 PM", "status": "read"}
# ]

# Parameters for the chat image
width, height = 500, 1000
background_color = "#F0F0F0"  # Lightest grey in Pillow's web-safe colors
sender_color = "lightgrey"
receiver_color = "white"
text_color = "black"
timestamp_color = "grey"
date_divider_color = "orange"

action_bar_height = 60  # Height of the top bar
action_bar_color = "white"
action_bar_text = "Steven"  # Or whatever title you want
action_bar_text_color = "black"
action_bar_padding = 10
icon_size = (40, 40)
icon_path = "profile.png"

# Font settings
# font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"  # Replace with your font path
# font_size = 20
# timestamp_font_size = 14

# Load fonts
# font = ImageFont.truetype(font_path, font_size)
# timestamp_font = ImageFont.truetype(font_path, timestamp_font_size)

# Initialize image
img = Image.new("RGB", (width, height), color=background_color)
draw = ImageDraw.Draw(img)

def draw_action_bar(draw):
    draw.rectangle(
        [(0, 0), (width, action_bar_height)],
        fill=action_bar_color
    )
    
    # Add the icon
    try:
        icon = Image.open(icon_path)
        # Resize icon while maintaining aspect ratio
        icon.thumbnail(icon_size)
        # Calculate vertical position to center the icon
        icon_y = (action_bar_height - icon.height) // 2
        # Paste the icon (handles transparency if PNG)
        if icon.mode == 'RGBA':
            img.paste(icon, (action_bar_padding, icon_y), icon)
        else:
            img.paste(icon, (action_bar_padding, icon_y))
            
        # Calculate text position after icon
        text_start_x = action_bar_padding + icon_size[0] + action_bar_padding
    except FileNotFoundError:
        # If no icon is found, start text from the left padding
        text_start_x = action_bar_padding
    
    # Draw the action bar text
    text_bbox = draw.textbbox((0, 0), action_bar_text)
    text_height = text_bbox[3] - text_bbox[1]
    text_y = (action_bar_height - text_height) // 2  # Vertically center
    
    draw.text(
        (text_start_x, text_y),
        action_bar_text,
        fill=action_bar_text_color
    )

# Function to draw a chat bubble
def draw_bubble(draw, text, sender=True, timestamp="", status=None, y_position=0):
    # Determine bubble size
    max_text_width = 280
    wrapped_text = textwrap.fill(text, width=30)
    # Deprecated textsize replace with textbbox
    bbox = draw.textbbox((0, 0), wrapped_text)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    bubble_width = min(max_text_width, text_width + 20)
    bubble_height = text_height + 40

    # Set position and color
    x_position = 20 if sender else width - bubble_width - 20
    bubble_color = sender_color if sender else receiver_color

    #action bar
    draw_action_bar(draw)

    # Draw bubble
    draw.rounded_rectangle((x_position, y_position, x_position + bubble_width, y_position + bubble_height), radius=15, fill=bubble_color)

    # Draw text
    draw.multiline_text((x_position + 10, y_position + 10), wrapped_text, fill=text_color)

    # Draw timestamp
    timestamp_position = (x_position + bubble_width - 50, y_position + bubble_height - 20)
    draw.text(timestamp_position, timestamp, fill=timestamp_color)

    # Draw status icon (e.g., check mark for read status)
    if status == "read":
        status_position = (x_position + bubble_width - 30, y_position + bubble_height - 20)
        draw.text(status_position, "✓", fill=timestamp_color)

    # Return the new y_position for the next message
    return y_position + bubble_height + 20

# Function to draw date divider
def draw_date_divider(draw, text, y_position):
    # Replace textsize with textbbox
    bbox = draw.textbbox((0, 0), text)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    divider_x_position = (width - text_width) // 2
    draw.text((divider_x_position, y_position), text, fill=date_divider_color)
    return y_position + text_height + 20

# Draw the chat conversation
current_y = 50 + action_bar_height  # Starting y position

for i in range(len(conversation_data)):
    # Initialize new image for each progressive conversation
    img = Image.new("RGB", (width, height), color=background_color)
    draw = ImageDraw.Draw(img)
    
    current_y = 50 + action_bar_height  # Reset starting y position
    
    # Draw conversation up to current message
    for item in conversation_data[:i+1]:
        sender = item["speaker"] == "Person 2"
        current_y = draw_bubble(
            draw, 
            item["message"], 
            sender=sender, 
            timestamp=item["timestamp"], 
            y_position=current_y
        )
    
    # Generate a unique timestamp for the folder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_name = f"conversation_images_{timestamp}"
    
    # Create output directory if it doesn't exist
    os.makedirs(folder_name, exist_ok=True)
    
    # Save the progressive conversation in the uniquely named folder
    img.save(f"{folder_name}/conversation_{i+1}.png")


### Movie Maker

In [17]:
!pip install moviepy
!pip install --upgrade decorator==4.4.2
!pip install ffmpeg --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import os
import moviepy.editor as mp
from moviepy.editor import ImageSequenceClip, AudioFileClip


In [19]:
# Directory where images are stored
output_video = f"{folder_name}/chat_conversation.mp4"

# Get files
audio_file = "trend.mp3"
image_files = sorted([os.path.join(folder_name, img) for img in os.listdir(folder_name) if img.endswith(".png")])

# Define video parameters
# Frames per second; adjust based on how fast you want the conversation to flow
fps = 1  
duration = len(image_files) / fps

# Create the video clip
clip = ImageSequenceClip(image_files, fps=fps)
clip = clip.set_duration(duration)

# Save the video
clip.write_videofile(output_video, codec="libx264")


Moviepy - Building video conversation_images_20241116_173856/chat_conversation.mp4.
Moviepy - Writing video conversation_images_20241116_173856/chat_conversation.mp4



Moviepy - Done !
Moviepy - video ready conversation_images_20241116_173856/chat_conversation.mp4


In [20]:
!pip install requests
!pip install dotenv
!pip install google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [32 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): fini

### Saving video to GCS
This step is necessary because instagram only allows videos hosted on GCS to be posted.

In [21]:
import os
from google.cloud import storage
from datetime import datetime
from google.oauth2.service_account import Credentials
import subprocess

def convert_to_9_16_ratio(input_video, output_video):
    """
    Convert video to 9:16 aspect ratio (1080x1920 for Instagram)
    """
    command = [
        'ffmpeg',
        '-y',
        '-i', input_video,
        '-vf', 'scale=1080:1920',
        '-c:a', 'copy',
        output_video
    ]
    
    try:
        subprocess.run(command, check=True)
        print(f"Successfully converted video to 9:16 ratio: {output_video}")
    except subprocess.CalledProcessError as e:
        print(f"Error converting video: {e}")

def upload_to_gcs(local_file_path, bucket_name = 'porygon-video-generation_cloudbuild'):
    """
    Upload a file to Google Cloud Storage and return the public URL.
    
    Args:
        local_file_path (str): Path to the local file
        bucket_name (str): Name of the GCS bucket
    
    Returns:
        str: Public URL of the uploaded file
    """
    try:
        credentials = Credentials.from_service_account_file(
            'key.json'  # Use the same credentials file as your Google Sheets setup
        )
        # Initialize the client
        storage_client = storage.Client(credentials=credentials)
        
        # Get the bucket
        bucket = storage_client.bucket(bucket_name)
        
        # Create a unique blob name using timestamp
        blob_name = f"reels/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{os.path.basename(local_file_path)}"
        blob = bucket.blob(blob_name)
        
        # Upload the file
        generation_match_precondition = 0
        blob.upload_from_filename(local_file_path, if_generation_match=generation_match_precondition)
        
        # Make the blob publicly accessible
        blob.make_public()
        
        # Get the public URL
        public_url = blob.public_url
        print(f"File uploaded successfully to: {public_url}")
        return public_url
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

# Usage (add this before uploading to Instagram)
input_video = f"chat_conversation.mp4"
instagram_video = f"conversation_instagram.mp4"
convert_to_9_16_ratio(input_video, instagram_video)


try:
    
    # Upload to GCS and get the URL
    public_url = upload_to_gcs(instagram_video)
    
except Exception as e:
    print(f"Failed to upload: {str(e)}")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enab

Error converting video: Command '['ffmpeg', '-y', '-i', 'chat_conversation.mp4', '-vf', 'scale=1080:1920', '-c:a', 'copy', 'conversation_instagram.mp4']' returned non-zero exit status 254.
File uploaded successfully to: https://storage.googleapis.com/porygon-video-generation_cloudbuild/reels/20241116_174052_conversation_instagram.mp4


### Post to IG

[Information needed for publishing reels using native IG](https://github.com/fbsamples/reels_publishing_apis/tree/main/insta_reels_publishing_api_sample?fbclid=IwZXh0bgNhZW0CMTEAAR1wf3OKOAHY3xp09E6USeHq7UJMblg9DK5oS0h31-_mYIx4qjGt4w1L9D4_aem_wWDShfiY8ppLdmrNlVMP9A)

[additional information on publishing reels for facebook pages use graph.instagram.com not facebook](https://developers.facebook.com/docs/video-api/guides/reels-publishing)

[Handling error codes and expired tokens](https://developers.facebook.com/docs/facebook-login/guides/access-tokens#generating-an-app-access-token)

In [8]:
import time
from datetime import datetime, timedelta
import requests
import os

def upload_to_instagram(video_path, caption):
    access_token = os.getenv('INSTAGRAM_TOKEN')
    
    # Get Instagram Business Account ID
    account_url = f"https://graph.instagram.com/v21.0/me?fields=user_id,username&access_token={access_token}"
    account_response = requests.get(account_url)
    # Debug: Print full response
    print("Account Response:", account_response.json())
    instagram_account_id = account_response.json()['user_id']
    
    # Step 1: Create container
    container_params = {
        'media_type': 'REELS',
        'video_url': public_url,
        'caption': caption,
        'access_token': access_token
    }
    container_url = f"https://graph.instagram.com/v21.0/{instagram_account_id}/media"
    
    container_response = requests.post(container_url, data=container_params)
    print("Container Response:",container_response.json())
    conatiner_id = container_response.json()['id']

    # Step 2: Check upload status and if the container is ready to publish
    
    # Set timeout to 5 minutes
    timeout = datetime.now() + timedelta(minutes=5)
    status = "IN_PROGRESS"
    
    while status == "IN_PROGRESS":
        if datetime.now() > timeout:
            raise TimeoutError("Upload status check timed out after 5 minutes")
            
        try:
            status_url = f"https://graph.instagram.com/v21.0/{conatiner_id}?fields=status&access_token={access_token}"
            status_response = requests.get(status_url)
            status_response.raise_for_status()  # Raise exception for non-200 status codes
            
            status_data = status_response.json()
            print("Status Response:", status_data)
            
            if "error" in status_data:
                raise Exception(f"API Error: {status_data['error'].get('message', 'Unknown error')}")
                
            status = status_data.get("status", "UNKNOWN")
            if status == "UNKNOWN":
                raise Exception("Could not determine upload status")
                
            if status != "IN_PROGRESS":
                break
                
            # Wait 5 seconds before checking again
            time.sleep(5)
            
        except requests.exceptions.RequestException as e:
            raise Exception(f"Network error while checking upload status: {str(e)}")
    
    # Step 3: Publish the container
    publish_url = f"https://graph.instagram.com/v21.0/{instagram_account_id}/media_publish"
    publish_params = {
        'creation_id': conatiner_id,
        'access_token': access_token
    }
    
    publish_response = requests.post(publish_url, data=publish_params)
    print(publish_response.text)
    
    # Return the URL of the posted content
    post_id = publish_response.json()['id']
    return f"https://www.instagram.com/p/{post_id}"

# Test the function
video_path = "conversation.mp4"  # Replace with your video path
caption = "Test upload from Python"

try:
    result = upload_to_instagram(video_path, caption)
    print(f"Successfully posted to Instagram: {result}")
except Exception as e:
    print(f"Failed to post to Instagram: {str(e)}")

Failed to post to Instagram: Expecting value: line 1 column 1 (char 0)


### Post to TikTok

https://developers.tiktok.com/doc/content-posting-api-get-started?enter_method=left_navigation

In [38]:
import os
import requests
import secrets
import urllib

auth_url = 'https://www.tiktok.com/v2/auth/authorize/'

#get the initial auth code
def generate_auth_url():
    csrf_token = secrets.token_hex(16)
    params = {
        'client_key': os.getenv('TIKTOK_CLIENT_KEY'),
        'redirect_uri': os.getenv('TIKTOK_REDIRECT_URI'),
        'response_type': 'code',
        'scope': 'video.publish',
        'state': csrf_token
    }
    url = auth_url + '?' + urllib.parse.urlencode(params)
    return url


# get auth token with our endpoint redirect url.
def get_tiktok_auth_token(code):
    open_url = 'https://open.tiktokapis.com/v2/oauth/token/'
    
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cache-Control': 'no-cache'
    }
    
    data = {
        'client_key': os.getenv('TIKTOK_CLIENT_KEY'),
        'client_secret': os.getenv('TIKTOK_CLIENT_SECRET'), 
        'code': code,
        'grant_type': 'authorization_code',
        'redirect_uri': os.getenv('TIKTOK_REDIRECT_URI')
    }
    
    response = requests.post(open_url, headers=headers, data=data)
    return response.json()


In [39]:
#what happens if we run the following multiple times witll the authurl mess up our code?
from IPython.display import IFrame, HTML, display

url = generate_auth_url()
print(url)
display(HTML(f'<script>window.open("{url}", "_blank");</script>'))


https://www.tiktok.com/v2/auth/authorize/?client_key=sbaweitjepifxk2nkg&redirect_uri=https%3A%2F%2Ftiktok.oligarchventures.com%2Fredirect&response_type=code&scope=video.publish&state=0deefa9f3f107678eb34f9aa1377588b


In [40]:
from IPython.display import clear_output

# This will create an input field
STOLEN_LINK_FROM_REDIRECT = input('Paste your code here: ')

# Optionally clear the input after getting the value
clear_output()
print(f"Received code: {STOLEN_LINK_FROM_REDIRECT}")

Received code: https://tiktok.oligarchventures.com/redirect?code=BHqN6dbfYLCqutC6M_klZuYSIAOtZCLJe26bJwS5iIJryryuD6U62EnFw0LhCzrud80u1ZLpN61RwqT3S9w7vGDd7CAbxVgWPoaa3KFrVd7jjkhT33sr-w6r4D6JL9YfkGIQlSBmbENWcg2YnD2S902b4FYdjF8hgeX6g_JNGEc_9ULHTQ8QkJk5JwNEr2jy%2A3%216312.va&scopes=user.info.basic%2Cvideo.publish&state=0deefa9f3f107678eb34f9aa1377588b


In [41]:
# the url we generate here is manually navigated to by the user and then the code is sent back to us by cutting from the search bar.  Not ideal.
# STOLEN_LINK_FROM_REDIRECT = 'https://tiktok.oligarchventures.com/redirect?code=NatJQbfOWhrqEM-Ig6SiPWUy77rQsBMbFnelhbEEtRE5B_yPlQlxeOW82u0dnUv50A9wcq-5XrGsS0M9wL7qVxNrnacp5m8c50S2KzAS7BXkyjJiexpScBDZiDVv7YG5OF8MNcBDbC2oIHk6m0nSJbByq2k4dlNs6yjbGzjzpjZGOjkio6c1DupJ2jYj5yih%2A3%216260.va&scopes=user.info.basic%2Cvideo.publish&state=a4f43458126f7e0e563827a3c988e919'
code = STOLEN_LINK_FROM_REDIRECT.split('code=')[1].split('&')[0]

json_response = get_tiktok_auth_token(code)
access_token = json_response['access_token']
refresh_token = json_response['refresh_token']
open_id = json_response['open_id']

print(access_token, refresh_token, open_id)

act.5kq9EdVaqd8dDVKQJaKqEGP6Hg5mh28hYQguYOcMxE83LUXnOzkRZ3rpfrWW!6307.va rft.ylpqrfnQJHULSP8IFBJF9rgo3DHNzbpFscjw0ysrqn1RFlg3LJFTPH6SkptP!6308.va -000Lc048FK60d7-9ZhAEX8kJboz4dl_VjfS


[learn more about 'chunking' and why it's not appropriate for teenagers](https://developers.tiktok.com/doc/content-posting-api-media-transfer-guide#)

In [42]:
def get_video_size(video_path):
    return os.path.getsize(video_path)

def init_video_upload(access_token, video_size, chunk_size=10000000):
    url = 'https://open.tiktokapis.com/v2/post/publish/video/init/'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json; charset=UTF-8'
    }
    
    # Videos under 5MB must be uploaded as one chunk
    if video_size < 5_000_000:
        chunk_size = video_size
    # Videos over 64MB must be chunked
    elif video_size > 64_000_000:
        chunk_size = 10_000_000  # Default 10MB chunks for large files
    # For videos between 5-64MB, use provided chunk size
    else:
        chunk_size = chunk_size
    total_chunks = -(-video_size // chunk_size)  # Ceiling division
    
    data = {
        "post_info": {
            "title": "this will be a funny #cat video on your @tiktok #fyp",
            "privacy_level": "SELF_ONLY", 
            "disable_duet": False,
            "disable_comment": True,
            "disable_stitch": False,
            "video_cover_timestamp_ms": 1000
        },
        "source_info": {
            "source": "FILE_UPLOAD",
            "video_size": video_size,
            "chunk_size": chunk_size,
            "total_chunk_count": total_chunks
        }
    }
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()

response_json = init_video_upload(access_token, get_video_size(instagram_video))
print(response_json)


{'data': {'publish_id': 'v_pub_file~v2-1.7437827438131857463', 'upload_url': 'https://open-upload-va.tiktokapis.com/upload?upload_id=7437827438131873847&upload_token=7a0a5ead-9201-5dde-feef-102f480f7c3b'}, 'error': {'code': 'ok', 'message': '', 'log_id': '2024111610504597C7ACE08AAC91070998'}}


KeyError: 'publish_id'

In [43]:
get_publish_id = response_json['data']['publish_id']
get_upload_url = response_json['data']['upload_url']
print(get_publish_id)
print(get_upload_url)

v_pub_file~v2-1.7437827438131857463
https://open-upload-va.tiktokapis.com/upload?upload_id=7437827438131873847&upload_token=7a0a5ead-9201-5dde-feef-102f480f7c3b


[Send the video](https://developers.tiktok.com/doc/content-posting-api-reference-direct-post#) from your local filesystem to the extracted upload_url using a PUT request. The video processing will occur asynchronously once the upload is complete.

In [50]:
def upload_video_chunk(upload_url, video_path):
    # Get file size for Content-Range header
    file_size = os.path.getsize(video_path)
    
    headers = {
        'Content-Type': 'video/mp4',
        'Content-Range': f'bytes 0-{file_size-1}/{file_size}'
    }
    
    with open(video_path, 'rb') as video_file:
        video_data = video_file.read()
        response = requests.put(upload_url, headers=headers, data=video_data)
        
    return response.status_code == 200

# Upload the video using the upload URL
upload_success = upload_video_chunk(get_upload_url, instagram_video)
print(f"Upload successful: {upload_success}")



null
Upload successful: False


[With the publish_id returned earlier, check for status updates using the Get Post Status endpoint.](https://developers.tiktok.com/doc/content-posting-api-reference-get-video-status)

In [51]:
def check_post_status(access_token, publish_id, max_attempts=30):
    url = 'https://open.tiktokapis.com/v2/post/publish/status/fetch/'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        'publish_id': publish_id
    }

    attempts = 0
    while attempts < max_attempts:
        response = requests.post(url, headers=headers, json=data)
        response_json = response.json()
        
        if 'data' not in response_json:
            print("Error: Unexpected response format")
            return response_json
            
        status = response_json['data'].get('status')
        
        if status == 'FAILED':
            print(f"Upload failed. Reason: {response_json['data'].get('fail_reason')}")
            return response_json
        elif status == 'PUBLISH_COMPLETE':
            print("Upload completed successfully!")
            return response_json
        elif status in ['PROCESSING_UPLOAD', 'PROCESSING_DOWNLOAD', 'SEND_TO_USER_INBOX']:
            print(f"Status: {status}. Uploaded bytes: {response_json['data'].get('uploaded_bytes', 0)}")
            time.sleep(2)  # Wait 2 seconds before checking again
        else:
            print(f"Unknown status: {status}")
            return response_json
            
        attempts += 1
    
    print("Maximum attempts reached. Upload status check timed out.")
    return response_json

# Poll for status updates with timeout
status_response = check_post_status(access_token, get_publish_id)
print(status_response)


Upload completed successfully!
{'data': {'status': 'PUBLISH_COMPLETE'}, 'error': {'code': 'ok', 'message': '', 'log_id': '20241116110046631A182A0521280ACB8B'}}
